In [1]:
import numpy as np
import pandas as pd
from load_data import loadVectors
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
x_train, y_train, x_validation, y_validation, x_test = loadVectors()

In [3]:
y_train = y_train - 1
y_validation = y_validation - 1;

In [4]:
y_train = keras.utils.to_categorical(y_train)
y_validation = keras.utils.to_categorical(y_validation)

In [5]:
model = keras.Sequential([
    keras.layers.Dense(4096, activation='relu', kernel_regularizer=keras.regularizers.l2(0.02), input_shape=(4096,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.004)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(29, activation='softmax')
])

In [6]:
model.compile(optimizer="adam", 
              loss='categorical_crossentropy', 
              metrics=['acc'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                131104    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 29)                957       
Total params: 16,913,373
Trainable params: 16,913,373
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_validation, y_validation))

Train on 5830 samples, validate on 2298 samples
Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor of shape [4096,32] and type float
	 [[Node: training/Adam/gradients/dense_2/weight_regularizer/Square_grad/Mul = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [4096,32] values: [0.008 0.008 0.008]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8), dpi=160)
plt.suptitle('Feature Extraction (no Augmentation)')
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predictions = model.predict(test)
predicted_class_indices=np.argmax(predictions,axis=1)
submission = pd.DataFrame({'id': np.arange(1,3461), 'label': predicted_class_indices})
submission.to_csv("submission.csv",index=False)